In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sqlite3
import pickle
pd.set_option('display.max_columns', None)

In [2]:
conn = sqlite3.connect('data\database.db')

In [3]:
#Train

query_train = '''
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2011
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2012
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2013
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2014
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2015
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2016
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2017
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2018
    GROUP BY "Month", "LSOA code"
    '''
dataframe2 = pd.read_sql_query(query_train, conn)

In [4]:
#Test

query_test = '''
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2019
    GROUP BY "Month", "LSOA code"
    '''

dataframe3 = pd.read_sql_query(query_test, conn)

In [5]:
#Train
crime_per_district = dataframe2.copy()

#Test
crime_per_district2 = dataframe3.copy()

In [6]:
#Train
crime_per_district["Falls within"] = crime_per_district["Falls within"].str.rsplit(' ',1).str[0]

#Test
crime_per_district2["Falls within"] = crime_per_district2["Falls within"].str.rsplit(' ',1).str[0]

In [7]:
#Train
crime_per_district = crime_per_district.rename(columns={'LSOA name': 'name', 'LSOA code': 'Geo code', 'Falls within':'Force Name'})

#Test
crime_per_district2 = crime_per_district2.rename(columns={'LSOA name': 'name', 'LSOA code': 'Geo code', 'Falls within':'Force Name'})

# Police

In [8]:
df_police = pd.read_excel("Data/Number of police officers per district per year.ods", engine = "odf")
df_police = df_police.rename(columns={'As at 31 March...': 'year', 'Geo code': 'Geo code police'})

#Train
bb = df_police[df_police['year']>2010]
cc = bb[bb['year']<2019]
aa = cc[cc['Worker type']=='Police Officer']
df_train = aa.groupby(['year','Force Name', 'Geo code police'])[['Total (headcount)']].sum()

#Test
ee = df_police[df_police['year']>2018]
ff = ee[ee['year']<2020]
dd = ff[ff['Worker type']=='Police Officer']
df_test = dd.groupby(['year','Force Name', 'Geo code police'])[['Total (headcount)']].sum()

# Population

In [9]:
df_population = pd.read_csv("data\Population of each district.csv")
df_population=df_population.rename(columns={'ladcode21':'Geo code', 'laname21':'name'})

df_districts = pd.read_csv("data\districts.csv")
df_districts = df_districts.rename(columns={'LAD19CD':'Geo code', 'LAD19NM':'name','PFA19CD': 'Geo code police', 'PFA19NM':'Force Name'})
df_districts.drop(['CSP19CD', 'CSP19NM', 'FID'], axis=1, inplace=True)

#Train
df_train2 = df_population.groupby(['name', 'Geo code'])[['population_2011', 'population_2012', 'population_2013', 'population_2014', 'population_2015', 'population_2016', 'population_2017', 'population_2018']].sum()
df_train2.reset_index(inplace=True)

abc_train = pd.merge(df_train2, df_districts, on='Geo code')
abc_train = abc_train.rename(columns={'name_x': 'name'})
abc_train.drop(['name_y'], axis=1, inplace=True)

#Test
df_test2 = df_population.groupby(['name', 'Geo code'])[['population_2019']].sum()
df_test2.reset_index(inplace=True)

abc_test = pd.merge(df_test2, df_districts, on='Geo code')
abc_test = abc_test.rename(columns={'name_x': 'name'})
abc_test.drop(['name_y'], axis=1, inplace=True)

# Merge population with the police

In [10]:
#Train
df_train3 = df_train.unstack(level=0)
df_train3.reset_index(inplace=True)

#Test
df_test3 = df_test.unstack(level=0)
df_test3.reset_index(inplace=True)

In [11]:
#Train
df_merged_train = pd.merge(abc_train, df_train3, on='Force Name')
df_merged_train.drop(('Geo code police', ''), axis=1, inplace=True)

#Test
df_merged_test = pd.merge(abc_test, df_test3, on='Force Name')
df_merged_test.drop(('Geo code police', ''), axis=1, inplace=True)

C:\Users\20193217\AppData\Local\Temp/ipykernel_11924/1921022594.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  df_merged_train = pd.merge(abc_train, df_train3, on='Force Name')
C:\Users\20193217\Anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\20193217\AppData\Local\Temp/ipykernel_11924/1921022594.py:6: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  df_merged_test = pd.merge(abc_test, df_test3, on='Force Name')


# Add year population and headcount column to merged dataframe

In [12]:
#Train
zz = df_merged_train.copy()
zz = zz.loc[zz.index.repeat(8)]
zz['year']= 2011
zz['population']= 61334
zz['headcount'] = 3181
zz.reset_index(inplace=True)

#Test
yy = df_merged_test.copy()
#yy = yy.loc[yy.index.repeat(1)]
yy['year']= 2019
yy['population']= 61334
yy['headcount'] = 3181
yy.reset_index(inplace=True)

In [13]:
zz.head()

,index,name,Geo code,population_2011,population_2012,population_2013,population_2014,population_2015,population_2016,population_2017,population_2018,Geo code police,Force Name,"(Total (headcount), 2011)","(Total (headcount), 2012)","(Total (headcount), 2013)","(Total (headcount), 2014)","(Total (headcount), 2015)","(Total (headcount), 2016)","(Total (headcount), 2017)","(Total (headcount), 2018)",year,population,headcount
0,0,Adur,E07000223,61334,61970,62605,63266,63526,63621,63721,63869,E23000033,Sussex,3181,3040,2928,2888,2897,2759,2688,2654,2011,61334,3181
1,0,Adur,E07000223,61334,61970,62605,63266,63526,63621,63721,63869,E23000033,Sussex,3181,3040,2928,2888,2897,2759,2688,2654,2011,61334,3181
2,0,Adur,E07000223,61334,61970,62605,63266,63526,63621,63721,63869,E23000033,Sussex,3181,3040,2928,2888,2897,2759,2688,2654,2011,61334,3181
3,0,Adur,E07000223,61334,61970,62605,63266,63526,63621,63721,63869,E23000033,Sussex,3181,3040,2928,2888,2897,2759,2688,2654,2011,61334,3181
4,0,Adur,E07000223,61334,61970,62605,63266,63526,63621,63721,63869,E23000033,Sussex,3181,3040,2928,2888,2897,2759,2688,2654,2011,61334,3181


In [14]:
yy.head()

,index,name,Geo code,population_2019,Geo code police,Force Name,"(Total (headcount), 2019)",year,population,headcount
0,0,Adur,E07000223,64301,E23000033,Sussex,2734,2019,61334,3181
1,1,Arun,E07000224,160758,E23000033,Sussex,2734,2019,61334,3181
2,2,Brighton and Hove,E06000043,290885,E23000033,Sussex,2734,2019,61334,3181
3,3,Chichester,E07000225,121129,E23000033,Sussex,2734,2019,61334,3181
4,4,Crawley,E07000226,112409,E23000033,Sussex,2734,2019,61334,3181


In [15]:
#Train
for i in range (1, zz.shape[0]):
    if zz['index'][i] == zz['index'][i-1]:
        zz['year'][i] = zz['year'][i-1]+1
        if zz['year'][i]==2011:
            zz['population'][i] = zz['population_2011'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2011)][i]
        if zz['year'][i]==2012:
            zz['population'][i] = zz['population_2012'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2012)][i]
        if zz['year'][i]==2013:
            zz['population'][i] = zz['population_2013'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2013)][i]
        if zz['year'][i]==2014:
            zz['population'][i] = zz['population_2014'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2014)][i]
        if zz['year'][i]==2015:
            zz['population'][i] = zz['population_2015'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2015)][i]
        if zz['year'][i]==2016:
            zz['population'][i] = zz['population_2016'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2016)][i]
        if zz['year'][i]==2017:
            zz['population'][i] = zz['population_2017'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2017)][i]
        if zz['year'][i]==2018:
            zz['population'][i] = zz['population_2018'][i]
            zz['headcount'][i] = zz[('Total (headcount)', 2018)][i]

#Test
# for j in range (1, yy.shape[0]):
#     if yy['index'][i] == yy['index'][i-1]:
# #        yy['year'][i] = yy['year'][i-1]+1
# #        if yy['year'][i]==2019:
#         yy['population'][i] = yy['population_2019'][i]
#         yy['headcount'][i] = yy[('Total (headcount)', 2019)][i]

C:\Users\20193217\AppData\Local\Temp/ipykernel_11924/1269433052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz['year'][i] = zz['year'][i-1]+1
C:\Users\20193217\AppData\Local\Temp/ipykernel_11924/1269433052.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz['population'][i] = zz['population_2012'][i]
C:\Users\20193217\AppData\Local\Temp/ipykernel_11924/1269433052.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz['headc

In [16]:
zz.drop(['population_2011', 'population_2012', 'population_2013','population_2014','population_2015','population_2016','population_2017', 'population_2018'], axis=1, inplace=True)
yy.drop(['population_2019'], axis=1, inplace=True)

In [17]:
zz.drop(['index',('Total (headcount)', 2011),('Total (headcount)', 2012), ('Total (headcount)', 2013), ('Total (headcount)', 2014), ('Total (headcount)', 2015), ('Total (headcount)', 2016),('Total (headcount)', 2017), ('Total (headcount)', 2018)], axis=1, inplace=True)
yy.drop(['index',('Total (headcount)', 2019)], axis=1, inplace=True)

# Add year column to dataframe from sql

In [18]:
#Train
crime_per_district['year'] = pd.to_datetime(crime_per_district['Month'], format='%Y-%m').dt.year

#Test
crime_per_district2['year'] = pd.to_datetime(crime_per_district2['Month'], format='%Y-%m').dt.year

# Data on Multiple Deprivation 

In [19]:
df_IMD = pd.read_csv("data\IMD_2015_LSOA_score.csv", skiprows = 7)

In [20]:
df_IMD = df_IMD[["Reference area", "2015"]]
df_IMD = df_IMD.rename(columns={"Reference area":"LSOA name", "2015" : "IMD score"})

# Merge everything together

In [21]:
#Train
df_merged_train2 = pd.merge(zz, crime_per_district, on=['year', 'Force Name'])

#Test
df_merged_test2 = pd.merge(yy, crime_per_district2, on=['year', 'Force Name'])

In [22]:
#Train
df_merged_train2 = df_merged_train2.rename(columns={'name_y': 'LSOA name', 'Geo code_y':'LSOA code', 'name_x':'District name', 'Geo code_x':'district code'})

#Test
df_merged_test2 = df_merged_test2.rename(columns={'name_y': 'LSOA name', 'Geo code_y':'LSOA code', 'name_x':'District name', 'Geo code_x':'district code'})

In [23]:
#Train
df_merged_train2['month'] = pd.to_datetime(df_merged_train2['Month'], format='%Y-%m').dt.month

#Test
df_merged_test2['month'] = pd.to_datetime(df_merged_test2['Month'], format='%Y-%m').dt.month

In [24]:
df_merged_test2.head()

,District name,district code,Geo code police,Force Name,year,population,headcount,Month,LSOA name,LSOA code,amount_of_crimes,month
0,Adur,E07000223,E23000033,Sussex,2019,61334,3181,2019-01,Brighton and Hove 026A,E01016849,10,1
1,Adur,E07000223,E23000033,Sussex,2019,61334,3181,2019-01,Brighton and Hove 029A,E01016850,28,1
2,Adur,E07000223,E23000033,Sussex,2019,61334,3181,2019-01,Brighton and Hove 029B,E01016851,7,1
3,Adur,E07000223,E23000033,Sussex,2019,61334,3181,2019-01,Brighton and Hove 026B,E01016852,15,1
4,Adur,E07000223,E23000033,Sussex,2019,61334,3181,2019-01,Brighton and Hove 026C,E01016853,16,1


In [25]:
df_merged_train2.head()

,District name,district code,Geo code police,Force Name,year,population,headcount,Month,LSOA name,LSOA code,amount_of_crimes,month
0,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 026A,E01016849,19,1
1,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 029A,E01016850,20,1
2,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 029B,E01016851,4,1
3,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 026B,E01016852,19,1
4,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 026C,E01016853,8,1


In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
#Train

In [28]:
lab = LabelEncoder()

labels_dist = df_merged_train2['District name'].unique()
lab.fit(labels_dist)
df_merged_train2[['District name']] = df_merged_train2[['District name']].apply(lab.fit_transform)

In [29]:
labels_dist = df_merged_train2['Force Name'].unique()
lab.fit(labels_dist)
df_merged_train2[['Force Name']] = df_merged_train2[['Force Name']].apply(lab.fit_transform)

In [30]:
labels_dist = df_merged_train2['LSOA code'].unique()
lab.fit(labels_dist)
df_merged_train2[['LSOA code']] = df_merged_train2[['LSOA code']].apply(lab.fit_transform)

In [31]:
#Test

In [32]:
labels_dist = df_merged_test2['District name'].unique()
lab.fit(labels_dist)
df_merged_test2[['District name']] = df_merged_test2[['District name']].apply(lab.fit_transform)

In [33]:
labels_dist = df_merged_test2['Force Name'].unique()
lab.fit(labels_dist)
df_merged_test2[['Force Name']] = df_merged_test2[['Force Name']].apply(lab.fit_transform)

In [34]:
labels_dist = df_merged_train2['LSOA code'].unique()
lab.fit(labels_dist)
df_merged_test2[['LSOA code']] = df_merged_test2[['LSOA code']].apply(lab.fit_transform)

In [35]:
df_t = df_merged_train2.copy()

In [36]:
df_merged_train2.head()

,District name,district code,Geo code police,Force Name,year,population,headcount,Month,LSOA name,LSOA code,amount_of_crimes,month
0,0,E07000223,E23000033,33,2011,61334,3181,2011-01,Brighton and Hove 026A,15724,19,1
1,0,E07000223,E23000033,33,2011,61334,3181,2011-01,Brighton and Hove 029A,15725,20,1
2,0,E07000223,E23000033,33,2011,61334,3181,2011-01,Brighton and Hove 029B,15726,4,1
3,0,E07000223,E23000033,33,2011,61334,3181,2011-01,Brighton and Hove 026B,15727,19,1
4,0,E07000223,E23000033,33,2011,61334,3181,2011-01,Brighton and Hove 026C,15728,8,1


In [37]:
final_data_training = pd.merge(df_merged_train2, df_IMD,  how='left', left_on=["LSOA name"], right_on = ["LSOA name"]).dropna()

In [38]:
final_data_test = pd.merge(df_merged_test2, df_IMD,  how='left', left_on=["LSOA name"], right_on = ["LSOA name"]).dropna()

In [53]:
final_data_test.head()

,District name,district code,Geo code police,Force Name,year,population,headcount,Month,LSOA name,LSOA code,amount_of_crimes,month,IMD score
0,0,E07000223,E23000033,33,2019,61334,3181,2019-01,Brighton and Hove 026A,15550,10,1,19.257
1,0,E07000223,E23000033,33,2019,61334,3181,2019-01,Brighton and Hove 029A,15551,28,1,22.759
2,0,E07000223,E23000033,33,2019,61334,3181,2019-01,Brighton and Hove 029B,15552,7,1,20.262
3,0,E07000223,E23000033,33,2019,61334,3181,2019-01,Brighton and Hove 026B,15553,15,1,30.256
4,0,E07000223,E23000033,33,2019,61334,3181,2019-01,Brighton and Hove 026C,15554,16,1,19.724


# Random forest implementation

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [40]:
features = df_merged_train2.drop(['amount_of_crimes', 'district code', 'Geo code police', 'Month', 'LSOA name'], axis=1)
features_test = df_merged_test2.drop(['amount_of_crimes', 'district code', 'Geo code police', 'Month', 'LSOA name'], axis=1)

X_train = features
y_train = df_merged_train2['amount_of_crimes']

X_test = features_test
y_test = df_merged_test2['amount_of_crimes']


In [41]:
features = final_data_training.drop(['amount_of_crimes', 'district code', 'Geo code police', 'Month', 'LSOA name'], axis=1)
features_test = final_data_test.drop(['amount_of_crimes', 'district code', 'Geo code police', 'Month', 'LSOA name'], axis=1)

X_train = features
y_train = final_data_training['amount_of_crimes']

X_test = features_test
y_test = final_data_test['amount_of_crimes']


In [42]:
cl = RandomForestRegressor(n_estimators = 10, random_state = 42)
cl.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [43]:
pred = cl.predict(X_test)

In [44]:
#Model evaluation
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test.values.ravel(), pred)

In [45]:
mae

8.84478363305743

In [46]:
from sklearn.metrics import r2_score
r2_score(y_test, pred)

0.005039210911921832

In [47]:
features

,District name,Force Name,year,population,headcount,LSOA code,month,IMD score
0,0,33,2011,61334,3181,15724,1,19.257
1,0,33,2011,61334,3181,15725,1,22.759
2,0,33,2011,61334,3181,15726,1,20.262
3,0,33,2011,61334,3181,15727,1,30.256
4,0,33,2011,61334,3181,15728,1,19.724
...,...,...,...,...,...,...,...,...
35973818,305,39,2018,498064,1021,30653,12,9.826
35973819,305,39,2018,498064,1021,30654,12,6.285
35973820,305,39,2018,498064,1021,30655,12,25.595
35973821,305,39,2018,498064,1021,30656,12,23.951


In [48]:
cl.feature_importances_

array([3.72795581e-04, 3.56944240e-02, 2.76048530e-02, 2.85296215e-03,
       3.51694877e-02, 3.60714561e-01, 6.05327303e-02, 4.77058187e-01])

In [49]:
from sklearn.metrics import mean_squared_error as MSE
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

RMSE :  17.404307


In [50]:
# xgboost

In [51]:
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE

In [52]:
# Instantiation
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 42)
  
# Fitting the model
xgb_r.fit(X_train, y_train)
  
# Predict the model
pred = xgb_r.predict(X_test)
  
# RMSE Computation
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

[13:46:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


NameError: name 'test_y' is not defined